In [1]:
import utils
import augment
import numpy as np
from model_03 import get_model
import losses
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ProgbarLogger, TensorBoard
from keras import backend as K
from keras import optimizers
from datetime import datetime
import matplotlib.pyplot as plt
import os
import metrics

Using TensorFlow backend.


In [2]:
img_size = 256
n_kernels = 16
p_valid = 0.15
d_train = "./data/{}/train/".format(img_size)
files = utils.get_files(d_train, verbose=False)
files = files
n_valid = int(len(files) * p_valid)
files_valid = files[:n_valid]
files_train = files[n_valid:]
d = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
f_model = "model_k{}_s{}_{}.h5".format(str(n_kernels), str(img_size), d)
X_train, Y_train = utils.load_data(files_train)
X_valid, Y_valid = utils.load_data(files_valid)

Loading data...
samples    301
Img size  ( 256, 256)
size      0.32 GB
Loading data...
samples     52
Img size  ( 256, 256)
size      0.05 GB


In [3]:
n_ones = np.sum(Y_train) + np.sum(Y_valid)
n_total = np.prod(Y_train.shape) + np.prod(Y_valid.shape)
n_zeros = n_total - n_ones
weights_zero = n_ones / n_total
weights_ones = n_zeros / n_total
weights_zero = 0.04
weights_ones = 0.96
print("Ones:  ", n_ones)
print("zeros: ", n_zeros)
print("Total: ", n_total)
print("1/T:   ", weights_zero)
print("0/T:   ", weights_ones)
print("n0 * w0 : n1 * w1 = {} : {}".format(n_ones * weights_ones, n_zeros * weights_zero))

Ones:   193767.0
zeros:  22940441.0
Total:  23134208
1/T:    0.04
0/T:    0.96
n0 * w0 : n1 * w1 = 186016.32 : 917617.64


In [4]:
model = get_model(img_size, n_kernels)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv_1_1 (Conv2D)               (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv_2_2 (Conv2D)               (None, 256, 256, 16) 2320        conv_1_1[0][0]                   
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 128, 128, 16) 0           conv_2_2[0][0]                   
__________________________________________________________________________________________________
conv2_1 (C

/home/matthias/projects/bio-unet/model_03.py:87: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input=inputs, output=conv10)


In [5]:
model.compile(loss=losses.make_bce_loss(weights_ones, weights_zero),
              optimizer=optimizers.SGD(lr=0.1, decay=1e-4, momentum=0.99, nesterov=True),
              metrics=['accuracy',
                       metrics.acc_zeros,
                       metrics.acc_ones,
                       metrics.mae_zeros,
                       metrics.mae_ones,
                       metrics.true_positives,
                       metrics.true_negatives,
                       metrics.false_positives,
                       metrics.false_negatives])
callbacks = [ModelCheckpoint("loss_" + f_model,
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1),
             TensorBoard(log_dir="logs/{}".format(d))
]

In [ ]:
#if os.path.isfile(f_model) and True:
#    model.load_weights(f_model)
hist = model.fit(x=X_train,
                 y=Y_train,
                 batch_size=1,
                 epochs=X_train.shape[0],
                 verbose=2,
                 callbacks=callbacks,
                 validation_data=(X_valid, Y_valid),
                 )

Train on 301 samples, validate on 52 samples
Epoch 1/301
 - 335s - loss: 0.0169 - acc: 0.9808 - acc_zeros: 0.9868 - acc_ones: 0.2736 - mae_zeros: 0.1304 - mae_ones: 0.5998 - true_positives: 0.0023 - true_negatives: 0.9785 - false_positives: 0.0131 - false_negatives: 0.0060 - val_loss: 0.0083 - val_acc: 0.9726 - val_acc_zeros: 0.9749 - val_acc_ones: 0.6995 - val_mae_zeros: 0.0520 - val_mae_ones: 0.3165 - val_true_positives: 0.0061 - val_true_negatives: 0.9665 - val_false_positives: 0.0249 - val_false_negatives: 0.0025

Epoch 00001: val_loss improved from inf to 0.00829, saving model to loss_model_k16_s256_05_07_2019_18_32_28.h5
Epoch 2/301
 - 343s - loss: 0.0081 - acc: 0.9686 - acc_zeros: 0.9705 - acc_ones: 0.7335 - mae_zeros: 0.0595 - mae_ones: 0.3031 - true_positives: 0.0063 - true_negatives: 0.9624 - false_positives: 0.0293 - false_negatives: 0.0021 - val_loss: 0.0061 - val_acc: 0.9726 - val_acc_zeros: 0.9737 - val_acc_ones: 0.8181 - val_mae_zeros: 0.0535 - val_mae_ones: 0.2444 - val


Epoch 00015: val_loss did not improve from 0.00269
Epoch 16/301
 - 310s - loss: 0.0033 - acc: 0.9823 - acc_zeros: 0.9828 - acc_ones: 0.8548 - mae_zeros: 0.0248 - mae_ones: 0.1829 - true_positives: 0.0077 - true_negatives: 0.9746 - false_positives: 0.0171 - false_negatives: 6.2044e-04 - val_loss: 0.0034 - val_acc: 0.9871 - val_acc_zeros: 0.9876 - val_acc_ones: 0.8711 - val_mae_zeros: 0.0198 - val_mae_ones: 0.2569 - val_true_positives: 0.0079 - val_true_negatives: 0.9791 - val_false_positives: 0.0123 - val_false_negatives: 6.2033e-04

Epoch 00016: val_loss did not improve from 0.00269
Epoch 17/301
 - 313s - loss: 0.0036 - acc: 0.9826 - acc_zeros: 0.9831 - acc_ones: 0.8487 - mae_zeros: 0.0264 - mae_ones: 0.1945 - true_positives: 0.0077 - true_negatives: 0.9749 - false_positives: 0.0167 - false_negatives: 6.5598e-04 - val_loss: 0.0034 - val_acc: 0.9704 - val_acc_zeros: 0.9703 - val_acc_ones: 0.9179 - val_mae_zeros: 0.0411 - val_mae_ones: 0.1139 - val_true_positives: 0.0084 - val_true_nega


Epoch 00030: val_loss did not improve from 0.00242
Epoch 31/301
 - 311s - loss: 0.0028 - acc: 0.9855 - acc_zeros: 0.9858 - acc_ones: 0.8715 - mae_zeros: 0.0198 - mae_ones: 0.1617 - true_positives: 0.0078 - true_negatives: 0.9776 - false_positives: 0.0140 - false_negatives: 4.9872e-04 - val_loss: 0.0026 - val_acc: 0.9882 - val_acc_zeros: 0.9886 - val_acc_ones: 0.8772 - val_mae_zeros: 0.0153 - val_mae_ones: 0.1850 - val_true_positives: 0.0081 - val_true_negatives: 0.9801 - val_false_positives: 0.0113 - val_false_negatives: 4.6715e-04

Epoch 00031: val_loss did not improve from 0.00242
Epoch 32/301
 - 309s - loss: 0.0026 - acc: 0.9856 - acc_zeros: 0.9860 - acc_ones: 0.8804 - mae_zeros: 0.0183 - mae_ones: 0.1565 - true_positives: 0.0079 - true_negatives: 0.9777 - false_positives: 0.0139 - false_negatives: 4.6385e-04 - val_loss: 0.0023 - val_acc: 0.9881 - val_acc_zeros: 0.9884 - val_acc_ones: 0.8868 - val_mae_zeros: 0.0148 - val_mae_ones: 0.1352 - val_true_positives: 0.0081 - val_true_nega

 - 309s - loss: 0.0016 - acc: 0.9904 - acc_zeros: 0.9905 - acc_ones: 0.9147 - mae_zeros: 0.0110 - mae_ones: 0.1120 - true_positives: 0.0082 - true_negatives: 0.9822 - false_positives: 0.0094 - false_negatives: 1.9461e-04 - val_loss: 0.0020 - val_acc: 0.9897 - val_acc_zeros: 0.9899 - val_acc_ones: 0.9113 - val_mae_zeros: 0.0115 - val_mae_ones: 0.1192 - val_true_positives: 0.0082 - val_true_negatives: 0.9815 - val_false_positives: 0.0100 - val_false_negatives: 2.9226e-04

Epoch 00046: val_loss improved from 0.00218 to 0.00203, saving model to loss_model_k16_s256_05_07_2019_18_32_28.h5
Epoch 47/301
 - 307s - loss: 0.0017 - acc: 0.9907 - acc_zeros: 0.9909 - acc_ones: 0.9204 - mae_zeros: 0.0112 - mae_ones: 0.1126 - true_positives: 0.0081 - true_negatives: 0.9826 - false_positives: 0.0090 - false_negatives: 2.1915e-04 - val_loss: 0.0025 - val_acc: 0.9890 - val_acc_zeros: 0.9894 - val_acc_ones: 0.8800 - val_mae_zeros: 0.0120 - val_mae_ones: 0.1286 - val_true_positives: 0.0080 - val_true_negat


Epoch 00061: val_loss did not improve from 0.00203
Epoch 62/301
 - 305s - loss: 0.0010 - acc: 0.9942 - acc_zeros: 0.9942 - acc_ones: 0.9331 - mae_zeros: 0.0067 - mae_ones: 0.0854 - true_positives: 0.0082 - true_negatives: 0.9860 - false_positives: 0.0057 - false_negatives: 1.2298e-04 - val_loss: 0.0037 - val_acc: 0.9897 - val_acc_zeros: 0.9903 - val_acc_ones: 0.8693 - val_mae_zeros: 0.0106 - val_mae_ones: 0.1409 - val_true_positives: 0.0078 - val_true_negatives: 0.9818 - val_false_positives: 0.0096 - val_false_negatives: 6.9780e-04

Epoch 00062: val_loss did not improve from 0.00203
Epoch 63/301
 - 300s - loss: 7.6035e-04 - acc: 0.9957 - acc_zeros: 0.9957 - acc_ones: 0.9445 - mae_zeros: 0.0051 - mae_ones: 0.0707 - true_positives: 0.0083 - true_negatives: 0.9874 - false_positives: 0.0043 - false_negatives: 8.6788e-05 - val_loss: 0.0065 - val_acc: 0.9935 - val_acc_zeros: 0.9947 - val_acc_ones: 0.7767 - val_mae_zeros: 0.0060 - val_mae_ones: 0.2402 - val_true_positives: 0.0073 - val_true_

...


Epoch 00109: val_loss did not improve from 0.00203
Epoch 110/301
 - 312s - loss: 6.8388e-04 - acc: 0.9960 - acc_zeros: 0.9960 - acc_ones: 0.9543 - mae_zeros: 0.0046 - mae_ones: 0.0628 - true_positives: 0.0083 - true_negatives: 0.9877 - false_positives: 0.0039 - false_negatives: 7.9336e-05 - val_loss: 0.0163 - val_acc: 0.9948 - val_acc_zeros: 0.9968 - val_acc_ones: 0.6974 - val_mae_zeros: 0.0036 - val_mae_ones: 0.3114 - val_true_positives: 0.0065 - val_true_negatives: 0.9883 - val_false_positives: 0.0032 - val_false_negatives: 0.0020

Epoch 00110: val_loss did not improve from 0.00203
Epoch 111/301
 - 309s - loss: 5.0001e-04 - acc: 0.9971 - acc_zeros: 0.9971 - acc_ones: 0.9637 - mae_zeros: 0.0033 - mae_ones: 0.0498 - true_positives: 0.0083 - true_negatives: 0.9888 - false_positives: 0.0029 - false_negatives: 4.6486e-05 - val_loss: 0.0169 - val_acc: 0.9945 - val_acc_zeros: 0.9964 - val_acc_ones: 0.7139 - val_mae_zeros: 0.0040 - val_mae_ones: 0.2942 - val_true_positives: 0.0067 - val_tru


Epoch 00125: val_loss did not improve from 0.00203
Epoch 126/301
 - 315s - loss: 2.8831e-04 - acc: 0.9982 - acc_zeros: 0.9982 - acc_ones: 0.9764 - mae_zeros: 0.0019 - mae_ones: 0.0313 - true_positives: 0.0083 - true_negatives: 0.9899 - false_positives: 0.0018 - false_negatives: 2.3268e-05 - val_loss: 0.0271 - val_acc: 0.9954 - val_acc_zeros: 0.9977 - val_acc_ones: 0.6602 - val_mae_zeros: 0.0025 - val_mae_ones: 0.3434 - val_true_positives: 0.0062 - val_true_negatives: 0.9891 - val_false_positives: 0.0023 - val_false_negatives: 0.0023

Epoch 00126: val_loss did not improve from 0.00203
Epoch 127/301
 - 313s - loss: 2.8387e-04 - acc: 0.9982 - acc_zeros: 0.9982 - acc_ones: 0.9752 - mae_zeros: 0.0019 - mae_ones: 0.0321 - true_positives: 0.0083 - true_negatives: 0.9899 - false_positives: 0.0017 - false_negatives: 2.2964e-05 - val_loss: 0.0268 - val_acc: 0.9953 - val_acc_zeros: 0.9975 - val_acc_ones: 0.6633 - val_mae_zeros: 0.0026 - val_mae_ones: 0.3400 - val_true_positives: 0.0063 - val_tru


Epoch 00141: val_loss did not improve from 0.00203
Epoch 142/301
 - 315s - loss: 2.5179e-04 - acc: 0.9984 - acc_zeros: 0.9984 - acc_ones: 0.9804 - mae_zeros: 0.0017 - mae_ones: 0.0273 - true_positives: 0.0083 - true_negatives: 0.9901 - false_positives: 0.0016 - false_negatives: 1.8047e-05 - val_loss: 0.0304 - val_acc: 0.9956 - val_acc_zeros: 0.9981 - val_acc_ones: 0.6364 - val_mae_zeros: 0.0021 - val_mae_ones: 0.3670 - val_true_positives: 0.0060 - val_true_negatives: 0.9895 - val_false_positives: 0.0019 - val_false_negatives: 0.0025

Epoch 00142: val_loss did not improve from 0.00203
Epoch 143/301
 - 324s - loss: 2.4878e-04 - acc: 0.9985 - acc_zeros: 0.9985 - acc_ones: 0.9796 - mae_zeros: 0.0016 - mae_ones: 0.0268 - true_positives: 0.0083 - true_negatives: 0.9901 - false_positives: 0.0015 - false_negatives: 1.9466e-05 - val_loss: 0.0310 - val_acc: 0.9955 - val_acc_zeros: 0.9980 - val_acc_ones: 0.6310 - val_mae_zeros: 0.0020 - val_mae_ones: 0.3716 - val_true_positives: 0.0060 - val_tru


Epoch 00157: val_loss did not improve from 0.00203
Epoch 158/301
 - 313s - loss: 2.3967e-04 - acc: 0.9985 - acc_zeros: 0.9985 - acc_ones: 0.9834 - mae_zeros: 0.0016 - mae_ones: 0.0253 - true_positives: 0.0083 - true_negatives: 0.9902 - false_positives: 0.0015 - false_negatives: 1.5918e-05 - val_loss: 0.0289 - val_acc: 0.9956 - val_acc_zeros: 0.9979 - val_acc_ones: 0.6634 - val_mae_zeros: 0.0022 - val_mae_ones: 0.3412 - val_true_positives: 0.0062 - val_true_negatives: 0.9894 - val_false_positives: 0.0021 - val_false_negatives: 0.0023

Epoch 00158: val_loss did not improve from 0.00203
Epoch 159/301
 - 313s - loss: 2.3481e-04 - acc: 0.9985 - acc_zeros: 0.9985 - acc_ones: 0.9834 - mae_zeros: 0.0015 - mae_ones: 0.0244 - true_positives: 0.0083 - true_negatives: 0.9902 - false_positives: 0.0014 - false_negatives: 1.5005e-05 - val_loss: 0.0299 - val_acc: 0.9956 - val_acc_zeros: 0.9980 - val_acc_ones: 0.6546 - val_mae_zeros: 0.0021 - val_mae_ones: 0.3500 - val_true_positives: 0.0061 - val_tru


Epoch 00173: val_loss did not improve from 0.00203
Epoch 174/301
 - 340s - loss: 2.1095e-04 - acc: 0.9987 - acc_zeros: 0.9987 - acc_ones: 0.9860 - mae_zeros: 0.0014 - mae_ones: 0.0212 - true_positives: 0.0083 - true_negatives: 0.9903 - false_positives: 0.0013 - false_negatives: 1.2319e-05 - val_loss: 0.0311 - val_acc: 0.9957 - val_acc_zeros: 0.9982 - val_acc_ones: 0.6461 - val_mae_zeros: 0.0019 - val_mae_ones: 0.3573 - val_true_positives: 0.0061 - val_true_negatives: 0.9897 - val_false_positives: 0.0018 - val_false_negatives: 0.0024

Epoch 00174: val_loss did not improve from 0.00203
Epoch 175/301
 - 343s - loss: 2.1092e-04 - acc: 0.9987 - acc_zeros: 0.9987 - acc_ones: 0.9869 - mae_zeros: 0.0014 - mae_ones: 0.0214 - true_positives: 0.0083 - true_negatives: 0.9903 - false_positives: 0.0013 - false_negatives: 1.2166e-05 - val_loss: 0.0305 - val_acc: 0.9957 - val_acc_zeros: 0.9981 - val_acc_ones: 0.6529 - val_mae_zeros: 0.0019 - val_mae_ones: 0.3500 - val_true_positives: 0.0061 - val_tru

In [ ]:
utils.plot(hist.history['loss'], hist.history['val_loss'], "Loss")
utils.plot(hist.history['acc'], hist.history['val_acc'], "ACC")
utils.plot(hist.history['acc_zeros'], hist.history['val_acc_zeros'], "ACC_zeros")
utils.plot(hist.history['acc_ones'], hist.history['val_acc_ones'], "ACC_ones")
utils.plot(hist.history['mae_zeros'], hist.history['val_mae_zeros'], "MAE_zeros")
utils.plot(hist.history['mae_ones'], hist.history['val_mae_ones'], "MAE_ones")
utils.plot(hist.history['true_negatives'], hist.history['val_true_negatives'], "TN")
utils.plot(hist.history['true_positives'], hist.history['val_true_positives'], "TP")
utils.plot(hist.history['false_negatives'], hist.history['val_false_negatives'], "FN")
utils.plot(hist.history['false_positives'], hist.history['val_false_positives'], "FP")

In [ ]:
model.save('model_k16_s256_whole_dataset.h5')